# Assignment 2
In this assignment, we will use a multi-layer perceptron network to build an image classifier for single digits. We will be using a public dataset for model development. The dataset we will be using is the MNIST digit dataset. The dataset contains 10 classes, where class `i` contains images of digit `i`.

In [17]:
import torch
import torch.nn as nn
import torchvision
from torchvision import datasets
import torch.utils.data as data
from torch.utils.data import Dataset, DataLoader, random_split
import numpy as np
import torchvision.datasets as datasets
import torchvision.transforms as transforms


####1. Create train_data and test_dataset objects using the MNIST digit dataset from torchvision.datasets module. (5 points)

In [26]:
transform = transforms.Compose([transforms.ToTensor()])
train_data = torchvision.datasets.MNIST(root='data', train=True, download=True, transform = transform)
test_data = torchvision.datasets.MNIST(root='data', train=False, download=True, transform = transform)

####2. Use the [random_split](https://pytorch.org/docs/stable/data.html#torch.utils.data.random_split) method to split the `train_data` into `train_dataset` (50000 images) and `validation_dataset` dataset (10000 images). (5 points)

In [27]:

# Split the train dataset into train_dataset and validation_dataset
train_dataset, validation_dataset = random_split(train_data, [50000, 10000])

print(len(train_dataset), len(validation_dataset))

50000 10000


#### 3. Create dataloader objects for `train_dataset`, `validation_dataset`, and `test_dataset`. (5 points)

In [28]:
train_loader = data.DataLoader(train_dataset, batch_size= 64, shuffle=True)
validation_loader = data.DataLoader(validation_dataset, batch_size=64, shuffle=True)
test_loader = data.DataLoader(test_data, batch_size=64, shuffle=False)

#### 4. Develop an MLP model for classifying MNIST images. The developed model should have four hidden layers of 256, 128, 64, and 32 neurons. Each hidden layer should be followed with a ReLU unit and a Dropout layer (p=0.2).  (15 points)

In [29]:

model = nn.Sequential(
    nn.Flatten(),
    nn.Linear(28*28, 256, bias=True),
    nn.ReLU(),
    nn.Dropout(0.2),
    nn.Linear(256, 128, bias=True),
    nn.ReLU(),
    nn.Dropout(0.2),
    nn.Linear(128, 64, bias=True),
    nn.ReLU(),
    nn.Dropout(0.2),
    nn.Linear(64, 32, bias=True),
    nn.ReLU(),
    nn.Dropout(0.2),
    nn.Linear(32, 10, bias=True),
)

#### 5. Define the components needed for training a deep learning model. (10 points)



In [44]:
import torch.optim as optim
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01,momentum=0.9)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

#### 6. Write the training loop and train the model for 100 epochs. Print the training and validation accuracy and loss for each epoch. (35 points)

In [53]:
model.train()
num_epochs = 100

for epoch in range(num_epochs):

    train_loss = 0
    correct = 0
    total = 0

    for images, labels in train_loader:
        # images = images.view(-1, 28*28)
        images = images.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    train_loss = train_loss / len(train_loader)
    train_acc = 100 * correct / total

    # Evaluate the model on validation dataset
    model.eval()
    val_loss = 0
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in validation_loader:
            images = images.view(-1, 28*28)
            outputs = model(images)
            loss = criterion(outputs, labels)

            val_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    val_loss = val_loss / len(validation_loader)
    val_acc = 100 * correct / total

    print(f"Epoch [{epoch+1}/{num_epochs}], Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}%, Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.2f}%")


Epoch [1/100], Train Loss: 0.0190, Train Acc: 99.41%, Val Loss: 0.1017, Val Acc: 98.20%
Epoch [2/100], Train Loss: 0.0012, Train Acc: 99.97%, Val Loss: 0.1021, Val Acc: 98.21%
Epoch [3/100], Train Loss: 0.0003, Train Acc: 100.00%, Val Loss: 0.1029, Val Acc: 98.23%
Epoch [4/100], Train Loss: 0.0001, Train Acc: 100.00%, Val Loss: 0.1086, Val Acc: 98.22%
Epoch [5/100], Train Loss: 0.0001, Train Acc: 100.00%, Val Loss: 0.1083, Val Acc: 98.24%
Epoch [6/100], Train Loss: 0.0001, Train Acc: 100.00%, Val Loss: 0.1076, Val Acc: 98.25%
Epoch [7/100], Train Loss: 0.0001, Train Acc: 100.00%, Val Loss: 0.1089, Val Acc: 98.25%
Epoch [8/100], Train Loss: 0.0001, Train Acc: 100.00%, Val Loss: 0.1100, Val Acc: 98.26%
Epoch [9/100], Train Loss: 0.0001, Train Acc: 100.00%, Val Loss: 0.1111, Val Acc: 98.25%
Epoch [10/100], Train Loss: 0.0001, Train Acc: 100.00%, Val Loss: 0.1120, Val Acc: 98.26%
Epoch [11/100], Train Loss: 0.0001, Train Acc: 100.00%, Val Loss: 0.1129, Val Acc: 98.26%
Epoch [12/100], Train

####6. Test the model using the `test_dataset`, and report accuracy and loss. (10 points)

In [48]:
def eval_model(model, test_loader, criterion): 
  model.eval()
  test_loss = 0
  correct = 0
  total = 0

  with torch.no_grad():
      for images, labels in test_loader:
          # images = images.view(-1, 28*28)
          images, labels = images.to(device), labels.to(device)
          outputs = model(images)
          loss = criterion(outputs, labels)

          test_loss += loss.item()
          _, predicted = torch.max(outputs.data, 1)
          total += labels.size(0)
          correct += (predicted == labels).sum().item()

  test_loss = test_loss / len(test_loader)
  test_acc = 100 * correct / total

  print(f"Test Loss: {test_loss:.4f}, Test Acc: {test_acc:.2f}%")

eval_model(model, test_loader, criterion)

Test Loss: 0.0936, Test Acc: 98.39%


#### 8. We would like to see how accurate the trained model is when applied to a set of images of digits with slight differences. The image of a digit in the MNIST dataset has a black background (0 value for pixel values). This might be like writing with white chalk on a blackboard. We make slight changes in the test datasets by applying a simple change `image = 1 - image`. In other words, we invert the pixel intensity values. The resulting images resemble a digit written with a black marker on a whiteboard. Test the model using the updated test_dataset, and report your observations regarding model performance. How would you change your pipeline if you redo this experiment? (15 points)

In [50]:
inv_transform = transforms.Compose([transforms.ToTensor(), lambda image : 1-image])
inv_test_data = torchvision.datasets.MNIST(root='data', train=False, download=True, transform = inv_transform)
inv_test_dataloader = data.DataLoader(inv_test_data, batch_size = 64, shuffle=True)

eval_model(model, inv_test_dataloader, criterion)

Test Loss: 9.7539, Test Acc: 7.29%


The accuracy of the model on the inverted dataset is 7.29%, which is very low compared to the accuracy of the non inverted dataset.

In [52]:
from random import random
new_transform = transforms.Compose([transforms.ToTensor(), lambda image : 1-image if random() < 0.5 else image])


After adding the new transformation, it would convert image randomly and would allow our model to be trained with these inverted images. This alllows our model to be trained on a wide variation of the mnist which would include random variations, hence making our model more accurate.
